# Fiber Network

The problem provided in this example is a fiber network with fixed-fixed (both displacement and moments) boundary conditions with a prescribed compressive displacement (i.e. nonhomogenous Dirichlet Boundary Condition) on the top boundary. Each fiber is modeled with 2D geometrically exact beams (i.e. Simo-Reissner Beams). For more information on beams [see here](../force_control/beam/README.md).



<img src="imgs/fiber.png" width="500">


In [1]:
%matplotlib inline
from dolfin import *
import numpy as np
import matplotlib.pyplot as plt
from ufl import diag, Jacobian, shape
from arc_length.displacement_control_solver import displacement_control # import displacement control formulation of arc-length solver


parameters["form_compiler"]["cpp_optimize"] = True
parameters["form_compiler"]["quadrature_degree"] = 3
parameters['reorder_dofs_serial'] = False

ffc_options = {"optimize": True, \
               "eliminate_zeros": True, \
               "precompute_basis_const": True, \
               "precompute_ip_const": True}

## Import Mesh and define function spaces
In the case of 2D beams we also define the rotation matrix about the $z$ axis and directional derivative with respect to the beam centerline.

In [2]:
mesh = Mesh()
with XDMFFile('./examples/displacement_control/voronoi.xdmf') as infile:
    infile.read(mesh)
    

Ue = VectorElement("CG", mesh.ufl_cell(), 2, dim=2) # displacement
Te = FiniteElement("CG", mesh.ufl_cell(), 1) # rotation


V = FunctionSpace(mesh, MixedElement([Ue, Te]))   

v_ = TestFunction(V)
u_, theta_ = split(v_)
dv = TrialFunction(V)
v = Function(V, name="Generalized displacement")
u, theta = split(v)

VR = TensorFunctionSpace(mesh, "DG", 0, shape=(2, 2))

V0 = FunctionSpace(mesh, "DG", 0)



Vu = V.sub(0).collapse()
disp = Function(Vu)

Jac = Jacobian(mesh)
gdim = mesh.geometry().dim()
Jac = as_vector([Jac[i, 0] for i in range(gdim)])
g01 = Jac/sqrt(dot(Jac, Jac))
g02 = as_vector([-g01[1],g01[0]])

r01 = outer(g01,as_vector([1,0]))
r02 = outer(g02, as_vector([0,1]))

R0 = r01+r02

#-----------------------Define Functions for beams-----------------------------------#
def tgrad(u): # directional derivative w.r.t. beam centerline
    return dot(grad(u), g01)

def rotation_matrix(theta): # 2D rotation matrix -- there is no need to do rotation parametrization for 2D beams
    return as_tensor([[cos(theta), -sin(theta)],[sin(theta), cos(theta)]])
Rot = rotation_matrix(theta)

## Define Dirichlet Boundary Conditions


**Note that for the case of displacement control, the FEniCS expression for the applied displacement mujst have be positive to prevent convergence issues.**

For example:

```
apply_disp = Expression("t", t = 0.0, degree = 0)
```
is valid and will not have convergence issues while
```
apply_disp = Expression("-t", t = 0.0, degree = 0)
```
can cause convergence issues.

The direction of applied loading will be determined by the initial load step.

In [3]:
H = 100.0
w = 100.0

def bottom(x, on_boundary):
    return near(x[1], 0, 1e-6) 
def top(x, on_boundary):
    return near(x[1], H,1e-6) 

def left(x, on_boundary):
    return near(x[0], 0,1e-6)
def right(x, on_boundary):
    return near(x[0], w,1e-6)


BC_bot = DirichletBC(V, Constant((0.0,0.0,0.0)), bottom) # fixed displacement and rotation
BC_top_x = DirichletBC(V.sub(0).sub(0), Constant(0.0), top) # fix displacement
BC_top_rot = DirichletBC(V.sub(1), Constant(0.0), top) # fix rotation

apply_disp = Expression("t", t=0.0, degree = 0) # Create expression to compress the top
BC_top_y = DirichletBC(V.sub(0).sub(1),apply_disp,top) # incrementally compress the top 

bcs = [BC_bot, BC_top_y, BC_top_rot, BC_top_x]

## Kinematics and Weak Form

In [4]:
# Kinematics: This is "total" beam formulation
defo = dot(R0.T,dot(Rot.T, g01 + tgrad(u)) - g01)
curv =  tgrad(theta)

In [5]:
# Geometrical properties
S = 1.5*3 # cross-sectional area
I = 3*1.5**3/12 # Area moment
G = 0.0412 # Shear Modulus
nu = 0.5
E = 2*G*(1+nu)

kappa = 5*(1+nu)/(6+5*nu) # Shear correction (Timoshenko)



# Stiffness moduli
ES = E*S
GS = G*kappa*S
EI = E*I

In [6]:
# Constitutive Equations
C_N = diag(as_vector([ES, GS]))

# Applied Load:
F_max = Constant((0.0,0.0))
M_max = Constant(0.0)

elastic_energy = 0.5 * (dot(defo, dot(C_N, defo)) + (EI*curv**2))*dx

F_int = derivative(elastic_energy, v, v_)
F_ext =(-M_max*theta_ + dot(F_max, u_)) * ds
residual = F_int - F_ext
tangent_form = derivative(residual, v, dv)

## Solver
To use our solver we first have to define the type of solver (i.e. displacement control or force control) and solver parameters before using the solver. Note that the correct type of solver has to first be imported (see first cell).
### Solver parameters
Here the parameters for both types of solvers:

>* `psi` : the scalar arc-length parameter. When psi = 1, the method becomes the spherical arc-length method and when psi = 0 the method becomes the cylindrical arc-length method
>* `abs_tol` *(optional)* : absolute residual tolerance for the linear solver (default value: 1e-10)
>* `rel_tol` *(optional)* :  relative residual tolerance for solver; the relative residual is defined as the ration between the current residual and initial residual (default value: DOLFIN_EPS)
>* `lmbda0` : the initial load parameter
>* `max_iter` : maximum number of iterations for the linear solver
>* `solver` *(optional)*: type of linear solver for the FEniCS linear solve function -- default FEniCS linear solver is used if no argument is used.

Aside from these solver parameters, the arguments need to solve the FEA problem must also be passed into the solver:
>* `u` : the solution function
>* `F_int` : First variation of strain energy (internal nodal forces)
>* `F_ext` : Externally applied load (external applied force)
>* `J` : The Jacobian of the residual with respect to the deformation (tangential stiffness matrix)
>* `displacement_factor` : The incremental displacement factor 

The solver can be called by:

`solver = force_control(psi,abs_tol,rel_tol,lmbda0,max_iter,u,F_int,F_ext,bcs,J,displacement_factor,solver)`

### Using the solver
1. Initialize the solver by calling solver.initialize()
2. Iteratively call solver.solve() until desired stopping condition

In [7]:
# Solver Parameters
psi = 1.0
abs_tol = 1.0e-6
lmbda0 = -0.5 # negative for compression
max_iter = 20
solver = 'mumps'

# Set up arc-length solver
solver = displacement_control(psi=psi, lmbda0=lmbda0, max_iter=max_iter, u=v,
                       F_int=F_int, F_ext=F_ext, bcs=bcs, J=tangent_form, displacement_factor = apply_disp, solver = solver)

In [ ]:
disp = [v.vector()[:]]
lmbda = [0]
# Function space to compute reaction force at each iteration
v_reac = Function(V)
bcRy = DirichletBC(V.sub(0).sub(1), Constant(1.0), bottom) # take reaction force from the bottom
f_reac = [0.0]
for ii in range(0,55):
    solver.solve()
    if solver.converged:
        # Store whole displacement field
        disp.append(v.vector()[:])
        # Store displacement factor
        lmbda.append(apply_disp.t)
        # Compute and store reaction force
        bcRy.apply(v_reac.vector())
        f_reac.append(assemble(action(residual,v_reac)))

Initializing solver parameters...
Starting initial Displacement Control Control with Newton Method:
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Iteration 0: | 
Absolute Residual: 2.7574e+00| Relative Residual: 1.0000e+00
Iteration 1: | 
Absolute Residual: 1.0849e-01| Relative Residual: 3.9345e-02
Iteration 2: | 
Absolute Residual: 9.9644e-03| Relative Residual: 3.6137e-03
Iteration

Iteration: 3 
|Total Norm: 3.9540e-09 |Residual: 8.0727e-13 |A: 3.9540e-09| Relative Norm : 1.3975e-06
Iteration: 4 
|Total Norm: 2.8773e-13 |Residual: 1.7632e-13 |A: -2.2737e-13| Relative Norm : 1.0169e-10

Arc-Length Step 18 :
Iteration: 0 
|Total Norm: 2.8955e-03 |Residual: 1.4819e-03 |A: -2.4875e-03| Relative Norm : 1.0000e+00
Iteration: 1 
|Total Norm: 1.6342e-01 |Residual: 7.1521e-05 |A: 1.6342e-01| Relative Norm : 5.6438e+01
Iteration: 2 
|Total Norm: 5.6206e-04 |Residual: 4.1324e-07 |A: 5.6206e-04| Relative Norm : 1.9412e-01
Iteration: 3 
|Total Norm: 5.8920e-08 |Residual: 3.2143e-11 |A: 5.8920e-08| Relative Norm : 2.0349e-05
Iteration: 4 
|Total Norm: 1.0359e-12 |Residual: 1.6213e-13 |A: 1.0232e-12| Relative Norm : 3.5778e-10

Arc-Length Step 19 :
Iteration: 0 
|Total Norm: 3.2065e-03 |Residual: 2.0757e-03 |A: -2.4440e-03| Relative Norm : 1.0000e+00
Iteration: 1 
|Total Norm: 1.7591e-01 |Residual: 3.2532e-05 |A: 1.7591e-01| Relative Norm : 5.4860e+01
Iteration: 2 
|Total Norm:

Iteration: 1 
|Total Norm: 9.5287e-01 |Residual: 3.9237e-05 |A: 9.5287e-01| Relative Norm : 1.7004e+02
Iteration: 2 
|Total Norm: 2.2671e-02 |Residual: 1.1405e-06 |A: 2.2671e-02| Relative Norm : 4.0456e+00
Iteration: 3 
|Total Norm: 2.3351e-07 |Residual: 3.0472e-12 |A: 2.3351e-07| Relative Norm : 4.1670e-05
Iteration: 4 
|Total Norm: 1.1069e-12 |Residual: 4.2224e-13 |A: -1.0232e-12| Relative Norm : 1.9752e-10

Arc-Length Step 34 :
Iteration: 0 
|Total Norm: 6.3787e-03 |Residual: 6.3216e-03 |A: -8.5147e-04| Relative Norm : 1.0000e+00
Iteration: 1 
|Total Norm: 7.3160e-01 |Residual: 3.4064e-05 |A: 7.3160e-01| Relative Norm : 1.1470e+02
Iteration: 2 
|Total Norm: 5.0131e-04 |Residual: 5.2939e-08 |A: 5.0131e-04| Relative Norm : 7.8592e-02
Iteration: 3 
|Total Norm: 2.0256e-09 |Residual: 4.1516e-13 |A: 2.0256e-09| Relative Norm : 3.1755e-07
Iteration: 4 
|Total Norm: 3.3325e-13 |Residual: 3.1326e-13 |A: -1.1369e-13| Relative Norm : 5.2245e-11

Arc-Length Step 35 :
Iteration: 0 
|Total Norm:

Iteration: 15 
|Total Norm: 5.4892e+01 |Residual: 2.0542e-03 |A: 5.4892e+01| Relative Norm : 2.2059e+03
Iteration: 16 
|Total Norm: 4.6070e+03 |Residual: 1.4889e-01 |A: 4.6070e+03| Relative Norm : 1.8514e+05
Iteration: 17 
|Total Norm: 1.5795e+04 |Residual: 9.6293e-02 |A: 1.5795e+04| Relative Norm : 6.3475e+05
Iteration: 18 
|Total Norm: 4.9339e+03 |Residual: 1.1649e-02 |A: 4.9339e+03| Relative Norm : 1.9828e+05
Iteration: 19 
|Total Norm: 2.2792e+04 |Residual: 1.3140e-01 |A: 2.2792e+04| Relative Norm : 9.1595e+05

Arc-Length Step 44 :
Iteration: 0 
|Total Norm: 9.2836e-03 |Residual: 9.2836e-03 |A: 3.3510e-05| Relative Norm : 1.0000e+00
Iteration: 1 
|Total Norm: 1.4605e+01 |Residual: 3.2691e-04 |A: 1.4605e+01| Relative Norm : 1.5732e+03
Iteration: 2 
|Total Norm: 1.1928e+05 |Residual: 7.9113e+00 |A: 1.1928e+05| Relative Norm : 1.2849e+07
Iteration: 3 
|Total Norm: 1.5155e+05 |Residual: 4.4117e+01 |A: 1.5155e+05| Relative Norm : 1.6325e+07
Iteration: 4 
|Total Norm: 8.0663e+04 |Residua

## Post Processing
Here we plot the final deformed shape and the equilibrium path.

In [ ]:
# Get dof coordinates:
x_dofs = V.sub(0).sub(0).dofmap().dofs()
y_dofs = V.sub(0).sub(1).dofmap().dofs()
theta_dofs = V.sub(1).dofmap().dofs()
dofs = V.tabulate_dof_coordinates()
dof_coords = dofs.reshape((-1, 2))

In [ ]:
x_nodal_coord = dof_coords[x_dofs][:,0]
y_nodal_coord = dof_coords[y_dofs][:,1]
# Get nodal values 

# Plot displacement field
disp_x = x_nodal_coord + disp[-1][x_dofs]
disp_y = y_nodal_coord + disp[-1][y_dofs]

plt.figure(figsize=(7,7))
plt.scatter(disp_x, disp_y, marker = '.', c = 'r', label = 'Deformed Configuration')
plt.scatter(x_nodal_coord,y_nodal_coord, marker = '.', c = 'k', alpha = 0.3, label = 'Initial Configuration')

plt.xlabel('x-coordinates')
plt.ylabel('y-coordinates')
plt.axis('equal')
plt.show()

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(-np.array(lmbda), f_reac, c='k', marker = 'o')
plt.xlabel('Applied Displacement')
plt.ylabel('Force')
plt.title('Equilibrium path')

## Optional: Creating an animation from solution snapshots

In [ ]:
from matplotlib import animation, rc

plt.rcParams["animation.html"] = "jshtml"

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111)

ax.set_xlim([0,w])
ax.set_ylim([-10,H+10])

deformed, = ax.plot([], [], lw = 7, c = 'r', label = 'Deformed Configuration', ls = 'None', marker = '.')
init, = ax.plot(x_nodal_coord, y_nodal_coord, c='k', lw = 5, ls = 'None', label = 'Initial Configuration', marker = '.', alpha = 0.3)
ax.legend(loc = 'lower right')

def drawframe(n):
    disp_x = x_nodal_coord + disp[n][x_dofs]
    disp_y = y_nodal_coord + disp[n][y_dofs]
    
    deformed.set_data(disp_x,disp_y)
    return deformed,

plt.close()
# blit=True re-draws only the parts that have changed.
anim = animation.FuncAnimation(fig, drawframe, frames=len(lmbda), interval=40, blit=True)

anim